In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from datetime import timezone
import numpy as np
import plotly.express as px

DSI Lab: Task 1

This excercise combines multiple phases of the CRISP-DM from Data Collection to Modelling and Model Evaluation. You should apply your knowledge on all the steps in the
process and also iterate through multiple steps.
The dataset is a collection of environmental sensor data from the H-Building of FH-Aachen.

## 1. Task a & b

Read 'MNIST.parquet'

In [ ]:
path = './datasets/07092021bimsim_room.parquet'
df = pd.read_parquet(path)
print(df.head(10))
print(df.info())
print(df.describe())